In [7]:
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from os.path import isdir
import numpy as np

In [8]:
def extract_images(filename, required_size=(200, 200)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    image = image.resize(required_size)
    pixels = asarray(image, dtype=np.float32)
    return pixels

In [9]:
def load_images(directory):
    images = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        img = extract_images(path)
        # store
        images.append(img)
    return images

In [10]:
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_images(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [11]:
trainX, trainy = load_dataset('xray_dataset_covid19/')

>loaded 100 examples for class: PNEUMONIA
>loaded 100 examples for class: NORMAL
>loaded 94 examples for class: Covid


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainy = le.fit_transform(trainy)

In [13]:
trainX.shape

(294, 200, 200, 3)

In [14]:
from sklearn.model_selection import train_test_split
trainX, testX, trainy, testy = train_test_split(trainX, trainy, test_size=0.5)

In [15]:
trainX.shape,trainy.shape,testX.shape,testy.shape

((147, 200, 200, 3), (147,), (147, 200, 200, 3), (147,))

# Convolutional Network

Now that we've gotten our feet we with a simple DNN, let's try something more advanced. Although the process is the same, we'll be working with some additional features:
- Convolution, pooling, and dropout layers for building more complex models
- Visualizing training with TensorBoard
- Validation and test set evaluation for measuring generalizability
- Exporting with SavedModel to save training progress and deploy trained models

In [2]:
import time
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}


import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers, metrics

In [45]:
CNN_model = tf.keras.Sequential([
    layers.Reshape(
        target_shape=[200, 200, 3],
        input_shape=(200, 200,3)),
    layers.Conv2D(2, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Conv2D(2, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Conv2D(2, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Conv2D(4, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Conv2D(4, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Conv2D(4, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Flatten(),
    layers.Dense(512, activation=tf.nn.relu),
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dropout(rate=0.4),
    layers.Dense(3)])

optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.5)

In [46]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [47]:
def train_step(model, optimizer, images, labels):

    # Record the operations used to compute the loss, so that the gradient
    # of the loss with respect to the variables can be computed.
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss = compute_loss(labels, logits)
        compute_accuracy(labels, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [48]:
def train(model, optimizer, dataset, log_freq=50):
    """
    Trains model on `dataset` using `optimizer`.
    """
    # Metrics are stateful. They accumulate values and return a cumulative
    # result when you call .result(). Clear accumulated values with .reset_states()
    avg_loss = metrics.Mean('loss', dtype=tf.float32)

    # Datasets can be iterated over like any other Python iterable.
    for images, labels in dataset:
        loss = train_step(model, optimizer, images, labels)
        avg_loss(loss)

        if tf.equal(optimizer.iterations % log_freq, 0):
            # summary_ops_v2.scalar('loss', avg_loss.result(), step=optimizer.iterations)
            # summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=optimizer.iterations)
            print('step:', int(optimizer.iterations),
                  'loss:', avg_loss.result().numpy(),
                  'acc:', compute_accuracy.result().numpy())
            avg_loss.reset_states()
            compute_accuracy.reset_states()

In [49]:
def test(model, dataset, step_num):
    """
    Perform an evaluation of `model` on the examples from `dataset`.
    """
    avg_loss = metrics.Mean('loss', dtype=tf.float32)

    for (images, labels) in dataset:
        logits = model(images, training=False)
        avg_loss(compute_loss(labels, logits))
        compute_accuracy(labels, logits)

    print('Model test set loss: {:0.4f} accuracy: {:0.2f}%'.format(
        avg_loss.result(), compute_accuracy.result() * 100))

    print('loss:', avg_loss.result(), 'acc:', compute_accuracy.result())
    # summary_ops_v2.scalar('loss', avg_loss.result(), step=step_num)
    # summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=step_num)

In [50]:
NUM_TRAIN_EPOCHS = 200

for i in range(NUM_TRAIN_EPOCHS):
    start = time.time()
    train_ds = tf.data.Dataset.from_tensor_slices((trainX, trainy))
    train_ds = train_ds.shuffle(200).batch(100)
    #   with train_summary_writer.as_default():
    train(CNN_model, optimizer, train_ds, log_freq=500)
    end = time.time()
    print('Train time for epoch #{} ({} total steps): {}'.format(
        i + 1, int(optimizer.iterations), end - start))
    test_ds = tf.data.Dataset.from_tensor_slices((testX, testy))
    test_ds = test_ds.batch(80)
    test(CNN_model, test_ds, optimizer.iterations)

Train time for epoch #1 (2 total steps): 3.8560259342193604
Model test set loss: 1.0904 accuracy: 36.39%
loss: tf.Tensor(1.0904262, shape=(), dtype=float32) acc: tf.Tensor(0.36394557, shape=(), dtype=float32)
Train time for epoch #2 (4 total steps): 4.786633014678955
Model test set loss: 1.0809 accuracy: 34.69%
loss: tf.Tensor(1.0808611, shape=(), dtype=float32) acc: tf.Tensor(0.3469388, shape=(), dtype=float32)
Train time for epoch #3 (6 total steps): 4.103327989578247
Model test set loss: 1.0580 accuracy: 37.07%
loss: tf.Tensor(1.0580037, shape=(), dtype=float32) acc: tf.Tensor(0.3707483, shape=(), dtype=float32)
Train time for epoch #4 (8 total steps): 5.34101414680481
Model test set loss: 1.0442 accuracy: 39.03%
loss: tf.Tensor(1.0441842, shape=(), dtype=float32) acc: tf.Tensor(0.39030612, shape=(), dtype=float32)
Train time for epoch #5 (10 total steps): 4.284988164901733
Model test set loss: 1.0232 accuracy: 41.22%
loss: tf.Tensor(1.0231919, shape=(), dtype=float32) acc: tf.Tenso

Model test set loss: 0.8014 accuracy: 57.96%
loss: tf.Tensor(0.8014352, shape=(), dtype=float32) acc: tf.Tensor(0.5795918, shape=(), dtype=float32)
Train time for epoch #41 (82 total steps): 3.8710761070251465
Model test set loss: 0.7914 accuracy: 58.11%
loss: tf.Tensor(0.7913521, shape=(), dtype=float32) acc: tf.Tensor(0.5811349, shape=(), dtype=float32)
Train time for epoch #42 (84 total steps): 3.887695789337158
Model test set loss: 0.8063 accuracy: 58.28%
loss: tf.Tensor(0.80628335, shape=(), dtype=float32) acc: tf.Tensor(0.5827664, shape=(), dtype=float32)
Train time for epoch #43 (86 total steps): 3.8976988792419434
Model test set loss: 0.8095 accuracy: 58.44%
loss: tf.Tensor(0.8094934, shape=(), dtype=float32) acc: tf.Tensor(0.5844012, shape=(), dtype=float32)
Train time for epoch #44 (88 total steps): 3.9059619903564453
Model test set loss: 0.9064 accuracy: 58.60%
loss: tf.Tensor(0.9064321, shape=(), dtype=float32) acc: tf.Tensor(0.58596164, shape=(), dtype=float32)
Train time 

Model test set loss: 0.8472 accuracy: 67.77%
loss: tf.Tensor(0.8471523, shape=(), dtype=float32) acc: tf.Tensor(0.6777319, shape=(), dtype=float32)
Train time for epoch #80 (160 total steps): 3.984592914581299
Model test set loss: 0.9174 accuracy: 68.00%
loss: tf.Tensor(0.9173987, shape=(), dtype=float32) acc: tf.Tensor(0.680017, shape=(), dtype=float32)
Train time for epoch #81 (162 total steps): 3.993367910385132
Model test set loss: 0.8994 accuracy: 68.22%
loss: tf.Tensor(0.89937246, shape=(), dtype=float32) acc: tf.Tensor(0.68216175, shape=(), dtype=float32)
Train time for epoch #82 (164 total steps): 4.039628744125366
Model test set loss: 0.8791 accuracy: 68.40%
loss: tf.Tensor(0.8791445, shape=(), dtype=float32) acc: tf.Tensor(0.6840053, shape=(), dtype=float32)
Train time for epoch #83 (166 total steps): 4.230347156524658
Model test set loss: 0.7969 accuracy: 68.60%
loss: tf.Tensor(0.79693246, shape=(), dtype=float32) acc: tf.Tensor(0.68600935, shape=(), dtype=float32)
Train tim

Model test set loss: 0.8831 accuracy: 73.19%
loss: tf.Tensor(0.8830513, shape=(), dtype=float32) acc: tf.Tensor(0.73186904, shape=(), dtype=float32)
Train time for epoch #119 (238 total steps): 3.8619110584259033
Model test set loss: 0.9147 accuracy: 73.33%
loss: tf.Tensor(0.91468346, shape=(), dtype=float32) acc: tf.Tensor(0.73326474, shape=(), dtype=float32)
Train time for epoch #120 (240 total steps): 3.8467679023742676
Model test set loss: 1.0827 accuracy: 73.44%
loss: tf.Tensor(1.082701, shape=(), dtype=float32) acc: tf.Tensor(0.7343537, shape=(), dtype=float32)
Train time for epoch #121 (242 total steps): 3.8873541355133057
Model test set loss: 0.9810 accuracy: 73.57%
loss: tf.Tensor(0.98099774, shape=(), dtype=float32) acc: tf.Tensor(0.73570585, shape=(), dtype=float32)
Train time for epoch #122 (244 total steps): 3.8583710193634033
Model test set loss: 1.0057 accuracy: 73.71%
loss: tf.Tensor(1.0056632, shape=(), dtype=float32) acc: tf.Tensor(0.7370637, shape=(), dtype=float32)


Model test set loss: 1.2834 accuracy: 77.10%
loss: tf.Tensor(1.2834141, shape=(), dtype=float32) acc: tf.Tensor(0.77098227, shape=(), dtype=float32)
Train time for epoch #158 (316 total steps): 3.913874864578247
Model test set loss: 1.3070 accuracy: 77.17%
loss: tf.Tensor(1.3069899, shape=(), dtype=float32) acc: tf.Tensor(0.7717429, shape=(), dtype=float32)
Train time for epoch #159 (318 total steps): 3.8081159591674805
Model test set loss: 1.5034 accuracy: 77.24%
loss: tf.Tensor(1.5034063, shape=(), dtype=float32) acc: tf.Tensor(0.77238697, shape=(), dtype=float32)
Train time for epoch #160 (320 total steps): 4.496499061584473
Model test set loss: 1.6811 accuracy: 77.30%
loss: tf.Tensor(1.6810734, shape=(), dtype=float32) acc: tf.Tensor(0.7730442, shape=(), dtype=float32)
Train time for epoch #161 (322 total steps): 5.471065998077393
Model test set loss: 1.7138 accuracy: 77.37%
loss: tf.Tensor(1.7138298, shape=(), dtype=float32) acc: tf.Tensor(0.7736933, shape=(), dtype=float32)
Train

Train time for epoch #196 (392 total steps): 3.784600019454956
Model test set loss: 2.0489 accuracy: 79.48%
loss: tf.Tensor(2.048857, shape=(), dtype=float32) acc: tf.Tensor(0.79475564, shape=(), dtype=float32)
Train time for epoch #197 (394 total steps): 3.8019001483917236
Model test set loss: 1.9716 accuracy: 79.53%
loss: tf.Tensor(1.9715587, shape=(), dtype=float32) acc: tf.Tensor(0.7952623, shape=(), dtype=float32)
Train time for epoch #198 (396 total steps): 3.8084049224853516
Model test set loss: 1.9433 accuracy: 79.58%
loss: tf.Tensor(1.9432611, shape=(), dtype=float32) acc: tf.Tensor(0.7957981, shape=(), dtype=float32)
Train time for epoch #199 (398 total steps): 3.9074811935424805
Model test set loss: 1.9599 accuracy: 79.63%
loss: tf.Tensor(1.959893, shape=(), dtype=float32) acc: tf.Tensor(0.7963286, shape=(), dtype=float32)
Train time for epoch #200 (400 total steps): 3.7992000579833984
Model test set loss: 2.0042 accuracy: 79.69%
loss: tf.Tensor(2.0041966, shape=(), dtype=fl

In [51]:
tf.saved_model.save(CNN_model,'CNN_Model_Covid_Pneumonia_Normal')

INFO:tensorflow:Assets written to: CNN_Model_Covid_Pneumonia_Normal/assets


# Inception Network

Here we train a tf.keras implementation of InceptionV3


![](inception.png)

In [16]:
class ConvBNRelu(keras.Model):
    
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()
        
        self.model = keras.models.Sequential([
            keras.layers.Conv2D(ch, kernelsz, strides=strides, padding=padding),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU()
        ])
        
        
    def call(self, x, training=None):
        
        x = self.model(x, training=training)
        
        return x 

In [17]:
class InceptionBlk(keras.Model):
    
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        
        self.ch = ch
        self.strides = strides
        
        self.conv1 = ConvBNRelu(ch, strides=strides)
        self.conv2 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_1 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_2 = ConvBNRelu(ch, kernelsz=3, strides=1)
        
        self.pool = keras.layers.MaxPooling2D(3, strides=1, padding='same')
        self.pool_conv = ConvBNRelu(ch, strides=strides)
        
        
    def call(self, x, training=None):
        
        
        x1 = self.conv1(x, training=training)

        x2 = self.conv2(x, training=training)
                
        x3_1 = self.conv3_1(x, training=training)
        x3_2 = self.conv3_2(x3_1, training=training)
                
        x4 = self.pool(x)
        x4 = self.pool_conv(x4, training=training)
        
        # concat along axis=channel
        x = tf.concat([x1, x2, x3_2, x4], axis=3)
        
        return x

In [18]:
class Inception(keras.Model):
    
    def __init__(self, num_layers, num_classes, init_ch=16, **kwargs):
        super(Inception, self).__init__(**kwargs)
        
        self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_layers = num_layers
        self.init_ch = init_ch
        
        self.conv1 = ConvBNRelu(init_ch)
        
        self.blocks = keras.models.Sequential(name='dynamic-blocks')
        
        for block_id in range(num_layers):
            
            for layer_id in range(2):
                
                if layer_id == 0:
                    
                    block = InceptionBlk(self.out_channels, strides=2)
                    
                else:
                    block = InceptionBlk(self.out_channels, strides=1)
                    
                self.blocks.add(block)
            
            # enlarger out_channels per block    
            self.out_channels *= 2
            
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)
        
        
    def call(self, x, training=None):
        
        out = self.conv1(x, training=training)
        
        out = self.blocks(out, training=training)
        
        out = self.avg_pool(out)
        out = self.fc(out)
        
        return out  

In [19]:
# build model and optimizer
batch_size = 32
epochs = 200
Inception_model = Inception(4, 3)
# derive input shape for every layers.
Inception_model.build(input_shape=(None, 200, 200, 3))
Inception_model.summary()

optimizer = keras.optimizers.Adam(learning_rate=1e-3)
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)

acc_meter = keras.metrics.Accuracy()

db_train = tf.data.Dataset.from_tensor_slices((trainX, trainy)).batch(256)
db_test = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(256)


for epoch in range(500):

    for step, (x, y) in enumerate(db_train):

        with tf.GradientTape() as tape:
            # print(x.shape, y.shape)
            # [b, 10]
            logits = Inception_model(x)
            # [b] vs [b, 10]
            loss = criteon(tf.one_hot(y, depth=3), logits)

        grads = tape.gradient(loss, Inception_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, Inception_model.trainable_variables))

        if step % 10 == 0:
            print(epoch, step, 'loss:', loss.numpy())


    acc_meter.reset_states()
    for x, y in db_test:
        # [b, 10]
        logits = Inception_model(x, training=False)
        # [b, 10] => [b]
        pred = tf.argmax(logits, axis=1)
        # [b] vs [b, 10]
        acc_meter.update_state(y, pred)

    print(epoch, 'evaluation acc:', acc_meter.result().numpy())

Model: "inception_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu_41 (ConvBNRelu) multiple                  512       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  5094624   
_________________________________________________________________
global_average_pooling2d_1 ( multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  1539      
Total params: 5,096,675
Trainable params: 5,091,843
Non-trainable params: 4,832
_________________________________________________________________
0 0 loss: 7.8019147
0 evaluation acc: 0.3537415
1 0 loss: 192.97495
1 evaluation acc: 0.2993197
2 0 loss: 66.97399
2 evaluation acc: 0.2993197
3 0 loss: 7.861458
3 evaluation acc: 0.2993197
4 0 loss: 1.7335942
4 evaluation acc: 0.2993197
5

KeyboardInterrupt: 

In [ ]:
tf.saved_model.save(Inception_model,'Inception_Model_Covid_Pneumonia_Normal')

# ResNet 

In [20]:
def conv3x3(channels, stride=1, kernel=(3, 3)):
    return keras.layers.Conv2D(channels, kernel, strides=stride, padding='same',
                               use_bias=False,
                            kernel_initializer=tf.random_normal_initializer())

In [21]:
class ResnetBlock(keras.Model):

    def __init__(self, channels, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()

        self.channels = channels
        self.strides = strides
        self.residual_path = residual_path

        self.conv1 = conv3x3(channels, strides)
        self.bn1 = keras.layers.BatchNormalization()
        self.conv2 = conv3x3(channels)
        self.bn2 = keras.layers.BatchNormalization()

        if residual_path:
            self.down_conv = conv3x3(channels, strides, kernel=(1, 1))
            self.down_bn = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training=None):
        residual = inputs

        x = self.bn1(inputs, training=training)
        x = tf.nn.relu(x)
        x = self.conv1(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)

        # this module can be added into self.
        # however, module in for can not be added.
        if self.residual_path:
            residual = self.down_bn(inputs, training=training)
            residual = tf.nn.relu(residual)
            residual = self.down_conv(residual)

        x = x + residual
        return x

In [22]:
class ResNet(keras.Model):

    def __init__(self, block_list, num_classes, initial_filters=16, **kwargs):
        super(ResNet, self).__init__(**kwargs)

        self.num_blocks = len(block_list)
        self.block_list = block_list

        self.in_channels = initial_filters
        self.out_channels = initial_filters
        self.conv_initial = conv3x3(self.out_channels)

        self.blocks = keras.models.Sequential(name='dynamic-blocks')

        # build all the blocks
        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):

                if block_id != 0 and layer_id == 0:
                    block = ResnetBlock(self.out_channels, strides=2, residual_path=True)
                else:
                    if self.in_channels != self.out_channels:
                        residual_path = True
                    else:
                        residual_path = False
                    block = ResnetBlock(self.out_channels, residual_path=residual_path)

                self.in_channels = self.out_channels

                self.blocks.add(block)

            self.out_channels *= 2

        self.final_bn = keras.layers.BatchNormalization()
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)

    def call(self, inputs, training=None):

        out = self.conv_initial(inputs)

        out = self.blocks(out, training=training)

        out = self.final_bn(out, training=training)
        out = tf.nn.relu(out)

        out = self.avg_pool(out)
        out = self.fc(out)


        return out

In [23]:
num_classes = 3
batch_size = 32
epochs = 200

# build model and optimizer
ResNet_model = ResNet([32, 16, 8], num_classes)
ResNet_model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
ResNet_model.build(input_shape=(None, 200,200,3))
print("Number of variables in the model :", len(ResNet_model.variables))
ResNet_model.summary()

y_ohe = tf.one_hot(trainy, depth=3).numpy()
testy_ohe = tf.one_hot(testy, depth=3).numpy()

# train
ResNet_model.fit(trainX, y_ohe, batch_size=batch_size, epochs=epochs,
          validation_data=(testX, testy_ohe), verbose=1)

# evaluate on test set
scores = ResNet_model.evaluate(testX, testy_ohe, batch_size, verbose=1)
print("Final test loss and accuracy :", scores)

Number of variables in the model : 577
Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_82 (Conv2D)           multiple                  432       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  1024000   
_________________________________________________________________
batch_normalization_196 (Bat multiple                  256       
_________________________________________________________________
global_average_pooling2d_2 ( multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  195       
Total params: 1,024,883
Trainable params: 1,018,611
Non-trainable params: 6,272
_________________________________________________________________
Train on 147 samples, validate on 147 samples
Epoch 1/200
147/147 [=====

KeyboardInterrupt: 

In [ ]:
tf.saved_model.save(ResNet_model,'ResNet_Model_Covid_Pneumonia_Normal')

# VGG16

In [24]:
from tensorflow.keras import regularizers

In [25]:
class VGG16(models.Model):


    def __init__(self, input_shape, num_classes):
        """

        :param input_shape: [32, 32, 3]
        """
        super(VGG16, self).__init__()

        weight_decay = 0.000
        self.num_classes = num_classes

        model = models.Sequential()

        model.add(layers.Conv2D(64, (3, 3), padding='same',
                         input_shape=input_shape, kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(self.num_classes))
        # model.add(layers.Activation('softmax'))


        self.model = model


    def call(self, x):

        x = self.model(x)

        return x

In [26]:
def compute_loss(logits, labels):
    return tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits, labels=labels))

In [27]:
VGG16_model = VGG16([200, 200, 3], 3)


# must specify from_logits=True!
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)
metric = keras.metrics.CategoricalAccuracy()

optimizer = optimizers.Adam(learning_rate=0.0001)

db_train = tf.data.Dataset.from_tensor_slices((trainX, trainy)).batch(256)
db_test = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(256)

for epoch in range(200):

    for step, (x, y) in enumerate(db_train):
        # [b, 1] => [b]
        # y = tf.squeeze(y, axis=1)
        # [b, 10]
        y = tf.one_hot(y, depth=3)

        with tf.GradientTape() as tape:
            logits = VGG16_model(x)
            loss = criteon(y, logits)
            # loss2 = compute_loss(logits, tf.argmax(y, axis=1))
            # mse_loss = tf.reduce_sum(tf.square(y-logits))
            # print(y.shape, logits.shape)
            metric.update_state(y, logits)

        grads = tape.gradient(loss, VGG16_model.trainable_variables)
        # MUST clip gradient here or it will disconverge!
        grads = [ tf.clip_by_norm(g, 15) for g in grads]
        optimizer.apply_gradients(zip(grads, VGG16_model.trainable_variables))

        if step % 40 == 0:
            # for g in grads:
            #     print(tf.norm(g).numpy())
            print(epoch, step, 'loss:', float(loss), 'acc:', metric.result().numpy())
            metric.reset_states()


    if epoch % 1 == 0:

        metric = keras.metrics.CategoricalAccuracy()
        for x, y in db_test:
            # [b, 1] => [b]
            # y = tf.squeeze(y, axis=1)
            # [b, 10]
            y = tf.one_hot(y, depth=3)

            logits = VGG16_model.predict(x)
            # be careful, these functions can accept y as [b] without warnning.
            metric.update_state(y, logits)
        print('test acc:', metric.result().numpy())
        metric.reset_states()

0 0 loss: 1.1591172218322754 acc: 0.3265306
test acc: 0.3469388
1 0 loss: 3.0694644451141357 acc: 0.33333334
test acc: 0.2993197
2 0 loss: 1.541196584701538 acc: 0.34013605
test acc: 0.30612245
3 0 loss: 1.079532504081726 acc: 0.34013605
test acc: 0.3537415
4 0 loss: 1.0796326398849487 acc: 0.33333334
test acc: 0.3809524
5 0 loss: 1.072597861289978 acc: 0.37414965
test acc: 0.3809524
6 0 loss: 1.0601069927215576 acc: 0.3877551
test acc: 0.5986394
7 0 loss: 1.0440694093704224 acc: 0.6530612
test acc: 0.65986395
8 0 loss: 1.0198856592178345 acc: 0.68707484
test acc: 0.63265306
9 0 loss: 0.9812823534011841 acc: 0.67346936
test acc: 0.67346936
10 0 loss: 0.9239126443862915 acc: 0.6938776
test acc: 0.82312924
11 0 loss: 0.835117518901825 acc: 0.89115644
test acc: 0.8367347
12 0 loss: 0.713874876499176 acc: 0.8027211
test acc: 0.59183675
13 0 loss: 0.8679106831550598 acc: 0.5986394
test acc: 0.3469388
14 0 loss: 1.5182301998138428 acc: 0.33333334
test acc: 0.6122449
15 0 loss: 0.786221802234

KeyboardInterrupt: 

In [28]:
tf.saved_model.save(VGG16_model,'VGG16_Model_Covid_Pneumonia_Normal')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: VGG16_Model_Covid_Pneumonia_Normal/assets
